<a href="https://colab.research.google.com/github/fabiominghetti/ia/blob/main/%5BRIWRS%5D_6_An%C3%A1lise_de_Sentimentos_Naive_Bayes-Original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carregar as bibliotecas e dados

## Bibliotecas

### Instalações via PIP

In [ ]:
!pip install unidecode
!pip install fractions
!pip install googletrans
!pip install ibm_watson
!pip install ibm_cloud_sdk_core
!pip install dotenv


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 5.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement fractions (from versions: none)
ERROR: No matching distribution found for fractions
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 985 kB/s 
     |████████████████████████████████| 42 kB 960 kB/s 
     |████████████████████████████████| 1.3 MB 11.5 MB/s 
     |████████████████████████████████| 65 kB 2.4 MB/s 
     |████████████████████████████████| 53 kB 995 kB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15735 sha256=00a2c25495ad38245629985342ff8694d237d8dd6f09998e28c93c8e6b13a752
  Stored in directory: /root/.cache/pip/wheels/20/da/eb/a

### Importações

In [ ]:
x# Load EDA Pkgs
import pandas as pd
import numpy as np
import random

# Load ML Pkgs
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('rslp')
from textblob import TextBlob

#charts & others stuff
from unidecode import unidecode
import matplotlib.pyplot as plt
import re
import string
from fractions import Fraction
import json
import os
from googletrans import Translator
translator = Translator()


# from dotenv import load_dotenv
# load_dotenv()
# load_dotenv('../ibm-credentials.env')

# IBM Watson
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson import LanguageTranslatorV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1  import Features, CategoriesOptions, SentimentOptions



# NATURAL_LANGUAGE_UNDERSTANDING_APIKEY = os.environ['NATURAL_LANGUAGE_UNDERSTANDING_APIKEY']
# NATURAL_LANGUAGE_UNDERSTANDING_URL    = os.environ['NATURAL_LANGUAGE_UNDERSTANDING_URL']

# LANGUAGE_TRANSLATOR_APIKEY = os.environ['LANGUAGE_TRANSLATOR_APIKEY']
# LANGUAGE_TRANSLATOR_URL    = os.environ['LANGUAGE_TRANSLATOR_URL']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


## Carrega o Arquivo

In [ ]:
def RoundSentiment(x):
  if x > 0.5:
    return 1
  elif x < -0.5:
    return -1
  return 0


df_o = pd.read_csv(f"https://raw.githubusercontent.com/lucianomcsilva/RIWRS/main/tweets_com_ibm_watson_e_ingles.csv")
df_o.drop(df_o.columns[df_o.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
#df[['id_str', 'text', 'screen_name', 'search_term', 'search_term_group', 'source', 'ibm_watson_sentiment', 'ibm_watson_sentiment_label']].sample(10)
df = df_o[['id_str', 'text', 'screen_name', 'search_term', 'search_term_group', 'source', 'ibm_watson_sentiment']].loc[df_o['search_term_group'].ne('star+')]
df['sentiment'] = df['ibm_watson_sentiment'].apply(lambda x: RoundSentiment(x))

# Naive Bayes

## Prepara uma amostra com 100 registros para validarmos a acuracia por nos mesmos

In [ ]:
# Faz uma amostra apenas com 200 registros aleatorios
# Iremos descatar alguns que classificarmos como neutro
df_nb = df.loc[df['sentiment'].ne(0)].sample(200)

#Salva o arquivo para podermos analisar na mao
df_nb.to_csv("https://raw.githubusercontent.com/lucianomcsilva/RIWRS/main/amostra.csv")

## Carrega o Arquivo Revisado

In [ ]:
df_nbr = pd.read_csv('https://raw.githubusercontent.com/lucianomcsilva/RIWRS/main/amostras-revisado.csv', delimiter=';')
df_nbr = df_nbr.loc[df_nbr['sentiment_revised'].ne(0)]
#df_nbr_pos = df_nbr.loc[df_nbr['sentiment_revised'].eq(1)]


In [ ]:
df_nbr

,Column1,id_str,text,screen_name,search_term,search_term_group,source,ibm_watson_sentiment,sentiment_revised,sentiment
1,3162,"1,45083E+18",Vou ter q assinar o globoplay só pra assistir ...,Gois99Leticia,globoplay,globoplay,Twitter for iPhone,"-8,91873E+15",-1,-1
2,2602,"1,45101E+18",PAREM DE ASSISTIR VERDADES SECRETAS O GLOBOPLA...,w_petun,globoplay,globoplay,Twitter for iPhone,"-9,24123E+15",-1,-1
3,5341,"1,45116E+18",enfim vendo death note antes de tirarem da net...,arigoncalvex,netflix,netflix,Twitter Web App,"-6,52531E+15",-1,-1
4,7659,"1,44949E+18",@nanacaligari @Netflix_Sandman @e3echo @lovegw...,n_ins,netflix,netflix,Twitter for iPhone,-728079,-1,-1
5,5776,"1,45087E+18",resmunguei horrores qd anunciaram rebelde da n...,_alinii,netflix,netflix,Twitter for iPhone,-760904,-1,-1
...,...,...,...,...,...,...,...,...,...,...
195,1474,"1,44991E+18",@GustavinPlus @Daniel_gutiere Nunca vi! Conto ...,cinestera,disney+,disney+,Twitter for iPhone,899835,-1,1
196,8012,"1,44957E+18",Assisti Agora Estamos Sozinhos na #PrimeVideo ...,crovislimone2,primevideo,primevideo,Twitter for iPhone,"-9,84106E+15",-1,-1
197,3284,"1,4524E+18",Como o aplicativo da HBO Max é ruim. Minha nossa!,r0cc0,hbo max,hbomax,Twitter for Android,-911342,-1,-1
198,2862,"1,45098E+18",O @globoplay de Alguém tá funcionando? #Verdad...,RealityTvBr,globoplay,globoplay,Twitter for Android,-700993,-1,-1


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics

vectorizer = CountVectorizer()

# Vectorize the text
X = vectorizer.fit_transform(df_nbr['text'])
Y = df_nbr['sentiment_revised']

# Split dataset
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.33,random_state=42)

vocabulary = vectorizer.get_feature_names()
dfX = pd.DataFrame(data=X.toarray(), columns=vocabulary) #.iloc[:,0::2]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# Building Our Model
clf = MultinomialNB()
clf.fit(x_train,y_train)

print("Accuracy of Model :",clf.score(x_test,y_test))

Accuracy of Model : 0.6724137931034483


In [ ]:
clf.get_params()

{'alpha': 1.0, 'class_prior': None, 'fit_prior': True}

In [ ]:
test_tweet = ['adorei! recomendo titans.',
              'Af pessimo',
              'netflix'
             ]
vect = vectorizer.transform(test_tweet).toarray()
vect.shape
clf.predict(vect)

array([ 1, -1, -1])

In [ ]:
# class_prob_sorted = clf.feature_log_prob_[0, :].argsort()[::-1]
# print(np.take(vectorizer.get_feature_names(), class_prob_sorted[:10]))

def get_salient_words(nb_clf, vect, class_ind):
    """Return salient words for given class
    Parameters
    ----------
    nb_clf : a Naive Bayes classifier (e.g. MultinomialNB, BernoulliNB)
    vect : CountVectorizer
    class_ind : int
    Returns
    -------
    list
        a sorted list of (word, log prob) sorted by log probability in descending order.
    """

    words = vect.get_feature_names_out()
    zipped = list(zip(words, nb_clf.feature_log_prob_[class_ind]))
    sorted_zip = sorted(zipped, key=lambda t: t[1], reverse=True)

    return sorted_zip

neg_salient_top_20 = get_salient_words(clf, vectorizer, 0)[:20]
pos_salient_top_20 = get_salient_words(clf, vectorizer, 1)[:20]



In [ ]:
neg_salient_top_20

[('que', -4.128374310808412),
 ('co', -4.236587895448645),
 ('https', -4.265575432321897),
 ('de', -4.357948752452912),
 ('netflix', -4.390738575275902),
 ('não', -4.613882126590113),
 ('eu', -4.700893503579742),
 ('disney', -4.796203683384068),
 ('da', -4.847496977771618),
 ('globoplay', -4.901564199041894),
 ('pra', -4.901564199041894),
 ('na', -5.019347234698277),
 ('com', -5.083885755835848),
 ('mas', -5.226986599476522),
 ('do', -5.489350863944012),
 ('vai', -5.489350863944012),
 ('pq', -5.594711379601838),
 ('um', -5.594711379601838),
 ('vou', -5.594711379601838),
 ('por', -5.712494415258222)]

In [ ]:
pos_salient_top_20

[('co', -4.28588277412098),
 ('https', -4.328442388539775),
 ('netflix', -4.328442388539775),
 ('de', -4.824879274853666),
 ('na', -4.898987247007389),
 ('que', -4.898987247007389),
 ('da', -5.066041331670554),
 ('com', -5.518026455413612),
 ('globoplay', -5.6721771352408705),
 ('meu', -5.6721771352408705),
 ('no', -5.6721771352408705),
 ('para', -5.6721771352408705),
 ('tem', -5.6721771352408705),
 ('as', -5.8544986920348245),
 ('do', -5.8544986920348245),
 ('está', -5.8544986920348245),
 ('eu', -5.8544986920348245),
 ('hbo', -5.8544986920348245),
 ('já', -5.8544986920348245),
 ('os', -5.8544986920348245)]